In [ ]:
![image.png](attachment:image.png)

In [ ]:
![image.png](attachment:image.png) ![image-2.png](attachment:image-2.png)

In [ ]:
# definitions

beam = {
    "Area"     : 9880  , #mm2
    "d"        : 450   , #mm
    "h"        : 378.8 , #mm
    "b_t"      : 190   , #mm
    "t_flange" : 14.6  , #mm
    "t_web"    : 9.4   , #mm
    "I_x"      : 33740 , #cm4
}

sac  = {
    "h_r"   : 50, #mm görseldeki h, hadve yüksekliği
    "w_r"   : 165, #mm
    "t_sac" : 1.2 #mm
}

anchorage = {
    "r_f" : 19, #mm
    "h_a" : 100, #mm
    "Fu"  : 450 #N/mm2
}

f_ck = 30 #N/mm2
Fy = 355 #N/mm2
Fu = 510 #N/mm2
Pg = 28 #kN
Pq = 10.5 #kN
L = 9 #m
wg = 1. #kN/m
h_c_üs = 80 #mm net beton örtüsü
h_c = h_c_üs + sac["h_r"]

In [27]:
def StudCheck(Ds : float, tf : float, Hs : float, IsWebAlignmentWelded : bool = False)-> bool:
    """_summary_

    Args:
        Ds (float): _description_
        tf (float): _description_
        Hs (float): Stud çivisi yüksekliği
        IsWebAlignmentWelded (bool, optional): _description_. Defaults to False.

    Returns:
        bool: _description_
    """

    if Ds > 2.5*tf or Ds > 19:
        print(f"Ds = {Ds}mm > 2.5*tf = {2.5*tf}mm X")
        print(f"Ds = {Ds}mm > 19mm X")
        return False
    if Hs < 4*Ds:
        print(f"Hs = {Hs}mm < 4*Ds = {4*Ds}mm X")
        return False
    if IsWebAlignmentWelded:
        print("Stud çivisi gövde hizasında bağlandığı için bu kontrollere gerek yoktur.")
        return True
    print(f"Ds = {Ds}mm ≤ 2.5*tf = {2.5*tf}mm √")
    print(f"Ds = {Ds}mm ≤ 19mm √")
    print(f"Hs = {Hs}mm ≥ 4*Ds = {4*Ds}mm √")
    return True

def StudSpaceCheck(Ds : float, s : float, Ycon : float) -> bool:
    """_summary_

    Args:
        Ds (float): _description_
        s (float): _description_
        Ycon (float): _description_

    Returns:
        bool: _description_
    """

    if s < 6*Ds or s > 8*Ycon or s >914:
        print(f"s = {s}mm < {6*Ds}mm X")
        print(f"s = {s}mm > {8*Ycon}mm X")
        print(f"s = {s}mm > 914mm X")
        return False
    print(f"s = {s}mm ≥ {6*Ds}mm √")
    print(f"s = {s}mm ≤ {8*Ycon}mm √")
    print(f"s = {s}mm ≤ 914mm √")
    return True

def DistanceBetweenTwoStudsCheck(x : float, Ds : float)-> bool:

    if x < 4*Ds:
        return False
    return True

def HadveUzerindeKalanStudBoyutlarıKontrolu(hr : float, t_sac : float, h_stud: float, Ycon : float) -> bool:
    """_summary_

    Args:
        hr (float): Hadve yüksekliği
        h_stud (float): Stud çivisi yüksekliği
        Ycon (float): Çelik kesit üst başlığından en üst beton lifine olan mesafe

    Returns:
        bool: _description_
    """
    if h_stud-hr-t_sac < 38 or Ycon - h_stud < 13 or Ycon - hr < 50:
        print(f"h_stud-hr-t_sac = {h_stud-hr-t_sac}mm < 38mm X")
        print(f"Ycon - h_stud-t_sac = {Ycon - h_stud-t_sac}mm < 13mm X")
        print(f"Ycon - hr = {Ycon - hr}mm < 50mm X")
        return False
    print(f"h_stud-hr-t_sac = {h_stud-hr-t_sac}mm ≥ 38mm √")
    print(f"Ycon - h_stud-t_sac = {Ycon - h_stud-t_sac}mm ≥ 13mm √")
    print(f"Ycon - hr = {Ycon - hr}mm ≥ 50mm √")
    return True

def MetalDeckCheck(hr : float, wr : float)-> bool:
    """_summary_

    Args:
        hr (float): Hadve yüksekliği
        wr (float): Hadve genişliği

    Returns:
        bool: _description_
    """
    if hr > 75 or wr < 50 :
        print(f"hr = {hr}mm > 75mm X")
        print(f"wr = {wr}mm < 50mm X")
        return False
    print(f"hr = {hr}mm ≤ 75mm √")
    print(f"wr = {wr}mm ≥ 50mm √")
    return True

def ConcAvailableStressCheck(f_ck : float) -> bool:
    if 20 <= f_ck and f_ck<70:
        print(f"20 N/mm^2 ≤ {f_ck} N/mm^2 < 70 N/mm^2 √")
        return True
    else:
        print(f"20 ≤ {f_ck} < 70 X")
        return False

def SteelAvailableStressCheck(Fy : float) -> bool:
    if Fy <= 460:
        print(f"{Fy} N/mm^2 ≤ 460 N/mm^2 √")
        return True
    else:
        print(f"Çelik sınıfı yönetmelik şartını sağlamamaktadır => {Fy} > {460} X")
        False

def ConcreteYoungModules(fck : float,wc : float = 2320)-> float:
    """_summary_

    Args:
        fck (float): MPa
        wc (float, optional): . Defaults to 2320 kg/m^3.

    Returns:
        float: _description_
    """
    Ec = 0.043 * wc **1.5 * fck**0.5
    return Ec

def RatioYoungModules(Ec : float,Es : float = 2*10**5)-> float:
    """_summary_

    Args:
        Ec (float): MPa
        Es (float, optional): MPa. Defaults to 2*10**5.
        

    Returns:
        float: _description_
    """
    return round(Es/Ec,3)

def ChengedConcToSteelArea(n : float, Ac)-> float:
    return round(Ac/n,3)

def CrushConcCapacity(fck : float, Ac : float)-> float:
    return round(0.85*fck*Ac,3)

def YieldBeamCapacity(fy : float, As : float)-> float:
    return round(fy*As,3)

def OneStudShearCapacity(Asa : float, fck : float, Ec : float, Rg : float, Rp : float, Fu : float = 448)-> float:
    Qn = 0.5 * Asa * (fck*Ec)**0.5
    TrashHold = Rg*Rp*Asa*Fu
    if Qn > TrashHold:
        Qn = TrashHold
    return round(TrashHold,3)

def ShearStudCapacity(N : int, Qn : float)-> float:
    """_summary_

    Args:
        N (int): Total shear stud number
        Qn (float): One stud shear capacity

    Returns:
        float: _description_
    """
    return round(N*Qn,3)

def EffectiveSlabWidth(L : float, Lu : float) -> float:
    """Efektif kompozit döşeme genişliği hesabı

    Args:
        L (float): Kiriş uzunluğu birim : m
        Lu (float): Kiriş aralığı birim : m

    Returns:
        b_eff float: efektif döşeme genişliği birim : m
    """
    b1 = L/8
    b2 = Lu/2

    b = 2*min(b1,b2)

    return b

def Calc_tc(hr : float, Ycon : float,IsParallel : bool)-> float:
    if IsParallel:
        tc = Ycon - (hr/2)
    else:
        tc = Ycon - hr

    return tc

def GetRp(IsPitchParaleltoBeam: bool=False)-> float:
    """Şekil verilmiş döşeme sacı için azaltma katsayısı

    Args:
        IsPitchParaleltoBeam (bool, optional): Döşeme sacı hadveleri kirişlere paralel ise True değilse False. Defaults to False.

    Returns:
        float: Rp değeri
    """
    if IsPitchParaleltoBeam:
        Rp = 0.75
    else:
        Rp = 0.6

    return Rp

def GetRg(StudsNumberInFlangeWidth : int, wr : float, hr : float, IsPitchParaleltoBeam: bool=False)-> float:
    """Şekil verilmiş döşeme sacı için azaltma katsayısı

    Args:
        StudsNumberInFlangeWidth (int): Kiriş başlık genişliğinde kaynaklanacak kayma çivisi adeti
        wr (float): Hadve genişliği
        hr (float): Hadve yüksekliği
        IsPitchParaleltoBeam (bool, optional): Döşeme sacı hadveleri kirişlere paralel ise True değilse False. Defaults to False.

    Returns:
        float: Rg değeri
    """
    ratio = wr/hr

    if IsPitchParaleltoBeam:
        if ratio < 1.5:
            Rg = 0.85
        else:
            Rg = 1.0
    else:
        if StudsNumberInFlangeWidth == 1:
            Rg = 1.0
        if StudsNumberInFlangeWidth == 2:
            Rg = 0.85
        if StudsNumberInFlangeWidth >= 3:
            Rg = 0.7
    
    return Rg

def Act(Ac : float, n : float)-> float:
    """_summary_

    Args:
        Ac (float): _description_
        n (float): Çelik elastisite modülünün beton elastisite modülüne oranı

    Returns:
        float: _description_
    """
    return Ac/n

def Calc_I_tr(Act : float,n : float,b_eff : float, hr : float, tc : float, hb : float,Ab : float, Ibeam : float)-> float:
    """_summary_

    Args:
        n (float): Çelik elastisite modülünün beton elastisite modülüne oranı
        b_eff (float): Efektif döşeme genişliği
        hr (float): Hadve yüksekliği
        tc (float): Beton döşeme yüksekliği
        hb (float): Kiriş yüksekliği
        Ab (float): Kiriş alanı
        Ibeam (float): Kiriş atalet momenti

    Returns:
        float: Dönüştürülmüş kesitin atalet momenti
    """

    Ict = b_eff * tc**3 / (12 * n)
    print(f"Ict = {Ict}")

    y_beam = tc + hr + hb/2
    y_conc = tc / 2
    print(f"y_beam = {y_beam}, y_conc ={y_conc}")

    TotalAy = Act * y_conc + Ab * y_beam
    TotalA = Act + Ab
    print(f"TotalAy = {TotalAy}, TotalA = {TotalA}")

    Y = TotalAy/TotalA
    print(f"Y ={Y}")

    d_conc = y_conc - Y
    d_beam = y_beam - Y
    print(f"d_conc ={d_conc}, d_beam = {d_beam}")

    IAd2_conc = Ict + (Act * d_conc**2)
    IAd2_beam = Ibeam + (Ab * d_beam**2)
    print(f"IAd2_conc = {IAd2_conc}, IAd2_beam = {IAd2_beam}")

    Itr = IAd2_beam + IAd2_conc

    return Itr

def Calc_Cf(Vbeam : float, Vcon : float):
    return min(Vbeam,Vcon)

def Calc_I_eff(Ibeam : float, TotalQn : float, Cf : float, Itr : float)-> float:
    ratiocomposite = (TotalQn / Cf)

    if ratiocomposite < 0.25:
        print(f"Kompozitlik oranı 0.25 değerinin altında olamaz stud çivisini arttırın veya daha yüksek dayanımlı bir stud çivisi seçin. {round(ratiocomposite,3)} < 0.25")
        return 0.0

    Ieff = Ibeam + ratiocomposite**0.5 * (Itr - Ibeam)

    return Ieff

def CompositeBeamDesignFlexuralCapacity(C_conc : float, T_steel : float, Ycon : float,
                                        BeamAs : float, BeamFy : float, Hbeam : float, t_flange : float, b_flange : float,
                                        Ac : float,hr : float,fck : float, beff : float, tc : float, t_web : float, fi_b : float = 0.9) -> float:

    C = min(C_conc,T_steel)
    a = C / (0.85 * fck * beff)
    print(f"a = {a} mm")
    Y2 = Ycon - (a/2)
    print(f"Y2 = {Y2} mm")

    if C_conc < T_steel : 
        print(f"Kompozit kiriş kısmi etkileşimlidir. Kompozitlik oranı %25'in altına inmemelidir.")

        A_steelcomp = (T_steel - C_conc) / (2*BeamFy)
        Y1 = round(A_steelcomp/b_flange,2)
        print(f"Y1(veya tf') = {Y1} mm")
        
        if Y1 < t_flange:
            print("PTE kiriş başlığındadır.")
            Csflange = Fy*b_flange*Y1
            print(f"Csflange = Fy*b_flange*Y1 = {Csflange} N")
            DesignMn = fi_b * ( (C_conc*(Y1+Y2)) + (2*Csflange*Y1/2) + (BeamAs*BeamFy*( (Hbeam/2) - Y1)) )

        if Y1 >= t_flange:
            print("PTE kiriş gövdesindedir.")
            Cflange = b_flange * t_flange * Fy
            print(f"Cflange = b_flange * t_flange * Fy = {Cflange} N")
            Y1 = ((T_steel - C_conc - 2*Cflange) / (2*t_web*Fy)) + t_flange
            print(f"Y1 = ((T_steel - C_conc - 2*Cflange) / (2*t_web*Fy)) + t_flange = {Y1}mm")
            Cweb    = t_web * Fy * (Y1 - t_flange)
            print(f"Cweb = t_web * Fy * (Y1 - t_flange) = {Cweb} N")
            DesignMn = fi_b * (C_conc*(Y1+Y2) + 2*Cflange*(Y1 - 0.5*t_flange) + 2*Cweb*(0.5*(Y1-t_flange)) + T_steel*(0.5*Hbeam-Y1))
            print(f"φMn = φ * (C_conc*(Y1+Y2) + 2*Cflange*(Y1 - 0.5*t_flange) + 2*Cweb*(0.5*(Y1-t_flange)) + T_steel*(0.5*Hbeam-Y1)) = {fi_b} * [{round(C_conc*(Y1+Y2),2)} + {round(2*Cflange*(Y1 - 0.5*t_flange),2)} + {round(2*Cweb*(0.5*(Y1-t_flange)),2)} + {round(T_steel*(0.5*Hbeam-Y1),2)}] = {round(DesignMn,3)} ")

    if C_conc >= T_steel :
        print(f"Kompozit kiriş tam etkileşimlidir. Plastik tarafsız eksen beton döşemenin içindedir.")
        DesignMn = fi_b * min(C_conc,T_steel) * ( 0.5*Hbeam + hr + tc - 0.5*a)
        print(f"φMn = φ * min(C_conc,T_steel) * ( 0.5*Hbeam + hr + tc - 0.5*a) = {fi_b} * {round(min(C_conc,T_steel),2)} *  {round( ( 0.5*Hbeam + hr + tc - 0.5*a),2)} = {round(DesignMn,3)}")

    return DesignMn

def calc_deflection(Beam : list,L : float , Pg : float,Pq : float,w_g : float = 1,E : float = 20_000):
    delta_1 = ((2 * Pg * L**3 * 10**3 / (28*E*Beam[6]*10**4)) + (5*w_g*L**4/(384*E*Beam[6]*10**4)))/10
    delta_2 = (2* Pq * L**3 * 10**3 / (28*E*Beam[6]*10**4))/10
    print(f"{delta_1} + {delta_2} = {delta_1+delta_2}")
    delta_yapım = delta_1 + delta_2

    return delta_yapım

def check_deflection(Beam : list,L : float , Pg : float,Pq : float,w_g : float = 1.,E : float = 20_000) :
    delta_1 = ((2 * Pg * L**3 * 10**3 / (28*E*Beam[6]*10**4)) + (5*w_g*L**4/(384*E*Beam[6]*10**4)))/10
    delta_2 = (2* Pq * L**3 * 10**3 / (28*E*Beam[6]*10**4))/10

    delta_yapım = delta_1 + delta_2

    if delta_1 > L/360:
        print(f"Sehim sınırı sağlanamadı => {delta_1}>{L/360}\nBu koşulun sağlanamadığı durumda çelik kirişe ters sehim verilmesi veya yapım aşamasında geçici destek kullanımı ile çelik kirişin düşey yerdeğiştirmesi azaltılabilir. (Bkz. Kaynak Yayınlar, AISC Design Guide 3).")
    if delta_yapım > L/300:
        print(f"Toplam sehim sınırı aşıldı kesit büyütülmeli veya mesnet koşulları değiştirilmeli.{delta_yapım} > {L/300} ")
    else:
        print(f"Sehim kontrolü sağlandı {delta_yapım} <= {L/300}. \nAyrıca, beton gerekli dayanımına ulaşıncaya kadar, \nçelik enkesitin yapım aşamasındaki sabit ve hareketli yükler altındaki eğilme dayanımı, \nYönetmelik Bölüm 9 da verilen kurallar gözönüne alınarak kontrol edilmelidir")

def yapim_asamasi_sehimi(Pg : float, Pq : float, L : float, Ix : float, E:float = 200_000, wg: float = 1.) -> float:
    """Basit mesnetli kompozit kiriş için yapım aşaması sehim hesabı

    Args:
        Pg (float): Ölü yük
        L (float): Kiriş uzunluğu
        Ix (float): Kiriş atalet momenti
        E (float, optional): Kiriş malzemesinin elastisite modülü. Defaults to 200_000. birim : N/mm2
        wg (float, optional): Kiriş ağırlığı birim:kN/m. Defaults to 1. 

    Returns:
        delta_1 (float): Yapım aşamasındaki sehim
    """

    #Ölü yük etkisi
    a = 2*(Pg * L**3 / (28 * E * Ix))
    b = (5 * wg * L**4 / (384 * E * Ix))
    delta_1 = a + b

    if delta_1 > (L / 360):
        print(f"Ölü yük durumunda sehim sınırı sağlanamadı => {round(delta_1,2)}mm > {round(L/360,2)}mm çelik kirişe ters sehim verilmesi veya yapım aşamasında geçici destek kullanımı ile çelik kirişin düşey yerdeğiştirmesi azaltılabilir.")

    #hareketli yük etkisi
    delta_2 = (2 * Pq * L **3 / (28 * E * Ix))

    #total etki
    delta = delta_1 + delta_2

    if delta > (L/300):
        print(f"Yapım aşaması sehim sınırı sağlanamadı kiriş boyutu arttırılabilir. => {round(delta,2)}mm > {round(L/300,2)}mm" )
    return round(delta,2)


In [28]:
# Ec = ConcreteYoungModules(fck=fck)
# n = RatioYoungModules(Ec=Ec)
# Rp = GetRp(IsPitchParaleltoBeam=False)
# Rg = GetRg(StudsNumberInFlangeWidth=1,wr=wr,hr=hr,IsPitchParaleltoBeam=False)
# Qn = OneStudShearCapacity(Asa=Asa, fck=fck, Ec=Ec, Rg=Rg, Rp=Rp, Fu=Fu)
# TotalQn = ShearStudCapacity(N=N, Qn=Qn)
# Itr = Calc_I_tr(n=n, b_eff=beff,hr=hr,hc=hc,hb=Hbeam,Ab=BeamAs,Ibeam=Ibeam)
# Cf = Calc_Cf(Vbeam=T_steel,Vcon=C_conc)
# Ieff = Calc_I_eff(Ibeam=Ibeam, TotalQn=TotalQn, Cf=Cf, Itr=Itr)

![compositebeamexample.png](compositebeamexample.png)

In [29]:
# Variables
f_ck     = 30     #N/mm2
Fy       = 355    #N/mm2
Abeam    = 33_420 #mm^2
Hbeam    = 800    #mm
Hweb     = 734    #mm
b_flange = 300    #mm
t_flange = 33     #mm
t_web    = 17.5   #mm
Laralık  = 4000   #mm
Lbeam    = 10000  #mm
Ycon     = 150    #mm Çelik kesit üst başlığından en üst beton lifine olan mesafe
hr       = 50     #mm


In [30]:
ConcAvailableStressCheck(f_ck=f_ck)

20 N/mm^2 ≤ 30 N/mm^2 < 70 N/mm^2 √


True

In [31]:
SteelAvailableStressCheck(Fy=Fy)

355 N/mm^2 ≤ 460 N/mm^2 √


True

In [32]:
b_eff = EffectiveSlabWidth(L=Lbeam,Lu=Laralık)
b_eff

2500.0

In [33]:
tc = Calc_tc(hr=hr,Ycon=Ycon,IsParallel=True)
tc

125.0

In [34]:
Ac = tc * b_eff
Ac

312500.0

In [35]:
Cs = YieldBeamCapacity(fy=Fy,As=Abeam)
Cs

11864100

In [36]:
Cc = CrushConcCapacity(fck=f_ck,Ac=Ac)
Cc

7968750.0

In [37]:
C = min(Cs,Cc)
C

7968750.0

In [39]:
DesignMn = CompositeBeamDesignFlexuralCapacity(C_conc=Cc, T_steel=Cs, Ycon=Ycon, 
                                    BeamAs=Abeam, BeamFy= Fy, Hbeam=Hbeam, t_flange=t_flange, b_flange=b_flange, 
                                    Ac=Ac, hr=hr,fck=f_ck, beff=b_eff, tc=tc, t_web=t_web)
DesignMn

a = 125.0 mm
Y2 = 87.5 mm
Kompozit kiriş kısmi etkileşimlidir.
Y1(veya tf') = 18.29 mm
PTE kiriş başlığındadır.
Csflange = Fy*b_flange*Y1 = 1947885.0 N


4866557841.135

![compositebeamexample2.png](compositebeamexample2.png)
![compositebeamexample2-1.png](compositebeamexample2-1.png)

In [40]:
# Variables
# hc' ==> tc
# hc ==> Ycon

# Beton bilgisi
f_ck     = 30     #N/mm2

# Çelik malzeme bilgisi
Fy       = 355    #N/mm2
Fu_beam  = 510    #N/mm2

# Çelik enkesit özellikleri
Abeam    = 9_880  #mm^2
Hbeam    = 450    #mm
Hweb     = 378.8  #mm
b_flange = 190    #mm
t_flange = 14.6   #mm
t_web    = 9.4    #mm
Ix       = 33_740*10**4 #mm^4
Ycon     = 130    #mm Çelik kesit üst başlığından en üst beton lifine olan mesafe

# Kiriş uzunluk ve aralık bilgisi
Laralık  = 7000   #mm
Lbeam    = 9000  #mm

# Stud çivisi bilgisi
D_stud   = 19
H_stud   = 100
Fu_stud  = 510    #N/mm2 çelik ankrajın minimum(karakteristik) çekme dayanımı

# Metal sac bilgisi
hr       = 50     #mm
wr       = 165    #mm
t_sac    = 1.2    #mm


In [41]:
ConcAvailableStressCheck(f_ck=f_ck)

20 N/mm^2 ≤ 30 N/mm^2 < 70 N/mm^2 √


True

In [42]:
SteelAvailableStressCheck(Fy=Fy)

355 N/mm^2 ≤ 460 N/mm^2 √


True

In [43]:
MetalDeckCheck(hr=hr,wr=wr)

hr = 50mm ≤ 75mm √
wr = 165mm ≥ 50mm √


True

In [44]:
HadveUzerindeKalanStudBoyutlarıKontrolu(hr=hr,t_sac=t_sac, h_stud=H_stud, Ycon=Ycon)

h_stud-hr-t_sac = 48.8mm ≥ 38mm √
Ycon - h_stud-t_sac = 28.8mm ≥ 13mm √
Ycon - hr = 80mm ≥ 50mm √


True

In [45]:
StudCheck(Ds=D_stud, tf=t_flange, Hs=H_stud, IsWebAlignmentWelded=False)

Ds = 19mm ≤ 2.5*tf = 36.5mm √
Ds = 19mm ≤ 19mm √
Hs = 100mm ≥ 4*Ds = 76mm √


True

In [46]:
b_eff = EffectiveSlabWidth(L=Lbeam,Lu=Laralık)
b_eff

2250.0

In [47]:
tc = Calc_tc(hr=hr,Ycon=Ycon,IsParallel=False)
tc

80

In [48]:
Ac = tc * b_eff
Ac

180000.0

In [49]:
Cs = YieldBeamCapacity(fy=Fy,As=Abeam)
Cs

3507400

In [50]:
Cc = CrushConcCapacity(fck=f_ck,Ac=Ac)
Cc

4590000.0

In [51]:
C = min(Cs,Cc)
C

3507400

In [53]:
DesignMn = CompositeBeamDesignFlexuralCapacity(C_conc=Cc, T_steel=Cs, Ycon=Ycon, 
                                    BeamAs=Abeam, BeamFy= Fy, Hbeam=Hbeam, t_flange=t_flange, b_flange=b_flange, 
                                    Ac = Ac, hr=hr,fck=f_ck, beff=b_eff, tc=tc, t_web=t_web)
DesignMn

a = 61.13115468409586 mm
Y2 = 99.43442265795207 mm
Kompozit kiriş tam etkileşimlidir. Plastik tarafsız eksen beton döşemenin içindedir.
φMn = φ * min(C_conc,T_steel) * ( 0.5*Hbeam + hr + tc - 0.5*a) = 0.9 * 3507400 *  324.43 = 1024129164.627


1024129164.627451